# Dialog State Tracking


## Intent Detection as Entailment
https://huggingface.co/facebook/bart-large-mnli
 

In [1]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer

nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')


In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
pp = classifier(sequence_to_classify, candidate_labels)

print(pp['sequence'])
print(pp['labels'])
print(pp['scores'])

Device set to use cuda:0


one day I will see the world
['travel', 'dancing', 'cooking']
[0.9938651323318481, 0.003273777896538377, 0.0028610406443476677]


In [3]:
# run through model pre-trained on MNLI
premise = sequence_to_classify
label = candidate_labels[0]

# we can use the first label as an example
hypothesis = f'This example is {label}.'

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                     truncation_strategy='only_first')
logits = nli_model(x)[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]

/opt/anaconda3/envs/nlp-cv-ir/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2708: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


In [4]:
prob_label_is_true

tensor([0.9945], grad_fn=<SelectBackward0>)

## Slot Filling as Zero-shot-QA
https://huggingface.co/deepset/roberta-base-squad2

In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch as torch

model_name = "deepset/roberta-base-squad2"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [6]:
prompt = "What are the vegetables that you want?"
user_u = "I'd like a salad with tomatos, lettuce and strawberries."

input_encoding = tokenizer.encode_plus(prompt, user_u, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
outputs = model(**input_encoding)


In [7]:
# Get the most likely beginning of answer with the argmax of the score
answer_start_scores = outputs.start_logits
answer_start = torch.argmax(answer_start_scores)

# Get the most likely end of answer with the argmax of the score
answer_end_scores = outputs.end_logits
answer_end = torch.argmax(answer_end_scores) + 1

print(answer_start)
print(answer_end)


tensor(17)
tensor(23)


In [8]:
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_encoding.input_ids[0][answer_start:answer_end]))

print(answer)


 tomatos, lettuce and strawberries


In [10]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

QA_input = {
    'question': prompt,
    'context': user_u
}
res = nlp(QA_input)
print(res)

Device set to use cuda:0


{'score': 0.8058444857597351, 'start': 22, 'end': 55, 'answer': 'tomatos, lettuce and strawberries'}


/opt/anaconda3/envs/nlp-cv-ir/lib/python3.12/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
